In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-04-20 01:27:20--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.09s   

2025-04-20 01:27:21 (11.4 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [4]:
with open(r"input.txt", 'r+') as f:
    text = f.read()

print(text[:10])

First Citi


In [10]:
unique_char = sorted(set(text))
unique_char = "".join(unique_char)
print(unique_char)
print(len(unique_char))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
# Building a simple tokenizer based on characters.

stoi = { ch:i for i, ch in enumerate(unique_char)}
itos = { i:ch for i, ch in enumerate(unique_char)}

print('a', stoi['a'], itos[stoi['a']])

# Encoding and decoding functions
encode = lambda s: [stoi[ch] for ch in s] # Takes a string and output integers
decode = lambda s: ''.join([itos[i] for i in s]) # Takes a list of integers as input and outpu a string

print(encode('hello'))
print(decode(encode('hello')))

a 39 a


In [20]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.size)
print(data[:10])

torch.Size([1115394]) <built-in method size of Tensor object at 0x7f4b8d144770>
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


In [23]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [24]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [28]:
# Eg

x = train_data[:block_size]
y = train_data[1:block_size+1]

for i in range(block_size):
    context = x[:i+1]
    target = y[i]

    print(context, target)

tensor([18]) tensor(47)
tensor([18, 47]) tensor(56)
tensor([18, 47, 56]) tensor(57)
tensor([18, 47, 56, 57]) tensor(58)
tensor([18, 47, 56, 57, 58]) tensor(1)
tensor([18, 47, 56, 57, 58,  1]) tensor(15)
tensor([18, 47, 56, 57, 58,  1, 15]) tensor(47)
tensor([18, 47, 56, 57, 58,  1, 15, 47]) tensor(58)


In [ ]:
torch.manual_seed(1337)

batch_size = 128
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    
    ix = torch.randint(0, len(data) - block_size, (batch_size,))            # Generating random posoitions and getting the data
    x = torch.stack([data[i:i+block_size] for i in ix])     # Stacking in rows
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    return x, y

xb, yb = get_batch('train')
print("inputs: ")
print(xb.shape)
print(xb)

print("Targets: ")
print(yb.shape)
print(yb)

inputs: 
torch.Size([128, 8])
tensor([[24, 43, 58,  ...,  1, 46, 43],
        [44, 53, 56,  ..., 46, 39, 58],
        [52, 58,  1,  ..., 39, 58,  1],
        ...,
        [43,  1, 54,  ..., 54, 50, 43],
        [53,  1, 47,  ..., 47, 58, 12],
        [39, 59, 52,  ..., 42,  1, 40]])
Targets: 
torch.Size([128, 8])
tensor([[43, 58,  5,  ..., 46, 43, 39],
        [53, 56,  1,  ..., 39, 58,  1],
        [58,  1, 58,  ..., 58,  1, 46],
        ...,
        [ 1, 54, 43,  ..., 50, 43,  5],
        [ 1, 47, 57,  ..., 58, 12,  0],
        [59, 52, 58,  ...,  1, 40, 63]])


In [38]:
# To see the batch

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(context, target)

tensor([24]) tensor(43)
tensor([24, 43]) tensor(58)
tensor([24, 43, 58]) tensor(5)
tensor([24, 43, 58,  5]) tensor(57)
tensor([24, 43, 58,  5, 57]) tensor(1)
tensor([24, 43, 58,  5, 57,  1]) tensor(46)
tensor([24, 43, 58,  5, 57,  1, 46]) tensor(43)
tensor([24, 43, 58,  5, 57,  1, 46, 43]) tensor(39)
tensor([44]) tensor(53)
tensor([44, 53]) tensor(56)
tensor([44, 53, 56]) tensor(1)
tensor([44, 53, 56,  1]) tensor(58)
tensor([44, 53, 56,  1, 58]) tensor(46)
tensor([44, 53, 56,  1, 58, 46]) tensor(39)
tensor([44, 53, 56,  1, 58, 46, 39]) tensor(58)
tensor([44, 53, 56,  1, 58, 46, 39, 58]) tensor(1)
tensor([52]) tensor(58)
tensor([52, 58]) tensor(1)
tensor([52, 58,  1]) tensor(58)
tensor([52, 58,  1, 58]) tensor(46)
tensor([52, 58,  1, 58, 46]) tensor(39)
tensor([52, 58,  1, 58, 46, 39]) tensor(58)
tensor([52, 58,  1, 58, 46, 39, 58]) tensor(1)
tensor([52, 58,  1, 58, 46, 39, 58,  1]) tensor(46)
tensor([25]) tensor(17)
tensor([25, 17]) tensor(27)
tensor([25, 17, 27]) tensor(10)
tensor([25